## Importing all necessary libararies

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
from Utility.ThesisUtility import getSentcesAndTokens,iterateAndAssignGoldStandard,printPairsEvaluation,printPurity

### Constants

In [3]:
#The file path to the dataset
dataFilePath = "../Dataset/Essays_all/10 essays for essays reference - ads/1"
evaluationFilePath = "../CSV Files/Clusters Evaluation results/GOLD vs Clusters - gold standards as centriodsWithTfIDf.xlsx"

### Reading the input(Essays) as text files and parsing it into sentences and tokens

In [4]:
sentences = getSentcesAndTokens(dataFilePath,False)

## Reading the gold standards and vectorizing them
df_ref = pd.read_excel("../CSV Files/Goldstandards/AD/Goldstandards.xlsx")
listRef = df_ref['Gold standard']

### building the sentences' vectors using TF-IDF vectorizer

In [5]:
vectorizer = TfidfVectorizer(min_df=1, stop_words='english')
vectorizer.fit_transform(sentences)
vectors = vectorizer.transform(sentences)

refs = vectorizer.transform(listRef)

### Iterating over all the sentneces and assigning the gold standard with the highest cosin similarity

In [ ]:
iterateAndAssignGoldStandard(vectors,sentences,refs,"Tf-Idf With Centriods.xlsx",True)

### Clusters Evaluation

In [ ]:
printPairsEvaluation(evaluationFilePath,"excel")

printPurity(evaluationFilePath,"excel")